<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/indexText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/indexText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/indexTitle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-42f1  GCE       3                                       RUNNING  us-central1-c


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
pip install nltk==3.7

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#remove mark
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#remove mark

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

ls: cannot access '/usr/lib/spark/jars/graph*': No such file or directory


In [ ]:
#remove mark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'corpus-big' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh': #todo change when we want to run on the original corpuse
        paths.append(full_path+b.name)


# #just for the training
# bucket_name = 'training_little_corpus'   
# full_path = f"gs://{bucket_name}/"

# paths = []
# client = storage.Client()
# blobs = client.list_blobs(bucket_name)
# for b in blobs:
#     if b.name != '_SUCCESS' and b.name != 'indexTitle.parquet/': #todo change when we want to run on the original corpuse
#         paths.append(full_path+b.name)



# Just needed in case you'd like to append it to an array
# paths = []

# local_download_path = "/content/new_training_bucket2.parquet"
# for filename in os.listdir(local_download_path):
#   paths.append(filename)
#   print(filename)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd #this is with text becuase we doint the inverted index for the title in this file 

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords) #to remark

################################################################################################################ we change it becuase nltk lib didn't work so we won't relate on it for now
# all_stopwords = corpus_stopwords 
################################################################################################################
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  hash={}


  for word in tokens:
    if word in all_stopwords:
      continue
    if word in hash:
      hash[word] = (id, hash[word][1]+1)
    else:
      hash[word] = (id,1)


  return list(hash.items())


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda x:x[0])


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  return postings.map( lambda x: (x[0], len(x[1])) )



def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  result = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey().map(lambda item: InvertedIndex.write_a_posting_list(item,bucket_name))  
  return result


In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='indexTitle'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
#check why not working and remark


# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'indexTitle')
# upload to gs
index_src = "indexTitles.pkl"
index_dst = f'gs://"indexes-my-proj"/{index_src}'
!gsutil cp $index_src $index_dst

CommandException: No URLs matched: indexTitle.pkl


In [ ]:
# index_src = "indexText.parquet" #this is writing the new corpus to a bucket in gcp
# index_dst = f'gs://{bucket_name}/{index_src}'
# parquetFile.write.parquet(index_dst)

In [ ]:
# !gsutil ls -lh $index_dst

 18.45 MiB  2022-12-08T06:53:07Z  gs://209206846/postings_gcp/index.pkl
TOTAL: 1 objects, 19342106 bytes (18.45 MiB)


In [ ]:
doc_text_pairs.toDF().show()

+--------------------+--------+
|                text|      id|
+--------------------+--------+
|'''Irvine''' () i...| 5201333|
|'''Lenore Skenazy...| 6236554|
|"'''Vitamin R (Le...| 6281833|
|'''More''' is an ...| 5892113|
|thumb|German coff...| 6332026|
|thumb| 200px |An ...|   21652|
|The '''Apple III'...|    2117|
| 

'''Luanda''' (...|   18402|
| 

'''Frankfurt''...|   10992|
|thumb|upright=1.3...|   30861|
|thumb|Overview of...|   16389|
|'''432 Park Avenu...|34374079|
|'''Cold brew''' g...|49099835|
|The '''Saturn Awa...|48530084|
|+ List of awards ...|49016960|
|thumb|Avocado toa...|53350936|
|thumb|Professor R...|52342272|
|'''Richard''' "''...|51759111|
|thumbtime=30|thum...|51430647|
|'''Novartis Inter...|  159284|
+--------------------+--------+
only showing top 20 rows

